In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog,milp,LinearConstraint
import itertools

### The scheduling problem and integer programming
In the previous section we looked at the assignment problem, which had a linear programming solution, but also a multiagent solution based on the idea of auctions and competitive equilibrium. Now we'll look at a more complicated scenario where multiple objects can be given to someone. The problem can still be stated as a linear program, but the answers will no longer be integers, hence the need for integer programming.

#### Definition

In a scheduling problem we have a set $N$ of $n$ agents, a set $X$ of $m$ discrete and consecutive time slots, a value function $v$ for assigning a combination of time slots to an agent, and a price $q$ to use each time slot for some other purpose (this is optional).

The value function says that a value of $w_i$ is generated if the number of timeslots assigned to agent $i$ is at least $\lambda_i$ (some measure of the time needed) before $d_i$ (the deadline).

Consider this problem, where we have 5 hour-long timeslots (9AM,10AM,11AM,12PM,1PM), and no alternative uses:

$
\begin{array}{cclc}
\text{agent} & \text{length}(\lambda) & \text{deadline}(d) & \text{value}(w) \\
\hline
\text{A} & 2 & \text{11 AM} & \$2 \\
\text{B} & 2 & \text{1 PM} & \$5 \\
\text{C} & 1 & \text{2 PM} & \$7 \\
\text{D} & 3 & \text{12 PM} & \$3 \\
\text{E} & 1 & \text{1 PM} & \$4 \\
\end{array}
$

This problem has a solution, which can be worked out by first putting $C$ at the end (as nothing else will be using that last slot) and then seeing out that of the other options the best is $B$ and $E$:

$
\begin{array}{lc}
\text{slot} & \text{user} \\
\hline
\text{9 AM} &  B\\
\text{10 AM} & B\\
\text{11 AM} & \\
\text{12 PM} & E \\
\text{1 PM} &  C \\
\end{array}
$

And there are other solutions, which are essentially just permutations of this. 

But how do we get there algorithmically?

First of all, the problem can be written as an integer program, where $S$ is the set of all possible subsets of $X$, and $x_{i,s}$ refers to the variable saying if agent $i$ was assigned to $s\in S$.

$$
\begin{align*}
\text{maximise} \quad & \sum_{s\in S, i\in N} v_i(s)x_{i,s} \\
\text{subject to} \quad & \sum_{s\in S} x_{i,s}<=1 \quad \forall i\in N \\
\text{} \quad & \sum_{s\in S: j\in X, i\in N} x_{i,s}<=1 \quad \forall j\in X \\
\end{align*}
$$

I.e., each person can only be assigned to 1 set, and each slot can only appear in one assignment. We can see this set up below. 

In [2]:
slots = ["9am","10am","11am","12pm","1pm"]
agents = ["A","B","C","D","E"]
lengths = np.array([2,2,1,3,1])
deadlines = np.array([1,3,4,2,3]) # index of the last slot that can be used
values = np.array([2,5,7,3,4])
S = [np.array(s) for r in range(1, len(slots)+1) for s in itertools.combinations(range(len(slots)), r)]

# get function values (s1a1, s1a2, s1a3, s1a4, s1a5 s2a1, ...)
v = np.zeros(len(S)*len(agents)).astype(int)
for s_index,s in enumerate(S):
    for a in range(len(agents)):
        length_used = np.sum(s<=deadlines[a])
        v[s_index*len(agents)+a] = values[a] if length_used>=lengths[a] else 0
        
constraints_A = []
constraints_b = []
# set constraint that variables are positive
for c in range(len(S)*len(agents)):
    constraints_A.append([-1 if g==c else 0 for g in range(len(S)*len(agents))])
    constraints_b.append(0)
    
# set constraint that an agent can only have 1 set
for agent_index in range(len(agents)):
    constraints_A.append([1 if agent_index==a else 0 for c in range(len(S)) for a in range(len(agents))])
    constraints_b.append(1)

# finally, set the constraint that an item can only appear once in an assigned set
for j in range(len(slots)):
    sets_with_j = [s_index for s_index,s in enumerate(S) if j in s]
    constraints_A.append([1 if c in sets_with_j else 0 for c in range(len(S)) for a in range(len(agents))])
    constraints_b.append(1)

A = np.array(constraints_A)
b = np.array(constraints_b)

We can try a linear program solver, but unfortunately this is more difficult than the first assignment problem, and there is no guarantee of integer solutions:

In [3]:
res = linprog(-v, A_ub=A, b_ub=b)
for ind in np.where(res["x"]>0)[0]:
    s_index = ind//len(agents)
    a_index = ind%len(agents)
    print("weight",round(res["x"][ind],2),"slots", [slots[j] for j in S[s_index]], "agent", agents[a_index])

weight 0.33 slots ['10am'] agent E
weight 0.67 slots ['12pm'] agent E
weight 1.0 slots ['1pm'] agent C
weight 0.67 slots ['9am', '11am'] agent B
weight 0.33 slots ['10am', '12pm'] agent B
weight 0.33 slots ['9am', '10am', '11am'] agent D


You could maybe try and fix this (i.e., turn these partial fractions into integers), but it seems like integer programming (expensive, not guaranteed to be optimal) is the proper solution:

In [4]:
constraints = LinearConstraint(A, -np.inf, b)
res = milp(c=-v, integrality=np.ones_like(v), constraints=constraints)
for ind in np.where(res["x"]>0)[0]:
    s_index = ind//len(agents)
    a_index = ind%len(agents)
    print("weight",round(res["x"][ind],2),"slots", [slots[j] for j in S[s_index]], "agent", agents[a_index])

weight 1.0 slots ['12pm'] agent E
weight 1.0 slots ['1pm'] agent C
weight 1.0 slots ['9am', '11am'] agent B


This solution is the same as above, except that $B$ is doing 11AM instead of 10AM.

#### A more general form of competitive equilibrium

Lets define what competitive equilibrium means. A solution $F$ is in competitive equilibrium for the prices $p$ if and only if:

- First point